In [18]:


import torch
import schnetpack as spk
import math
from schnetpack.datasets import QM9
import os

import numpy as np
from numpy import savetxt

from ase.io import read

qm9data = QM9('../../data/QM9/qm9.db', download=False, remove_uncharacterized=True)
split_file='../../data/trained_models/schnet_qm9energy_1000030/split.npz'
checkpoint_path = '../../data/trained_models/schnet_qm9energy_1000030/trained.pth'
n_atom_basis=30
n_filters=30
n_gaussians=20
n_interactions=3
cutoff = 4.

name_data = 'rep'


name = 'meoh'
molec_input_path = '../../data/datasets/extendchain/%s.xyz' %(name)
atoms = read(molec_input_path)
number_atoms = 6

print(atoms[1])

Atom('O', [-0.012792213, -0.6912656, 0.03563598], index=1)


In [15]:
def hook(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global layer
    #Update the embedding_output variable to be equal to our output tensor
    layer=out_tensor 

In [16]:

# Load split file 
train, val, test = spk.data.train_test_split(qm9data,split_file=split_file)

# Load atom ref data 
atomrefs = qm9data.get_atomref(QM9.U0)

# Define SchNet representation model

schnet = spk.representation.SchNet(
n_atom_basis=n_atom_basis, n_filters=n_filters, n_gaussians=n_gaussians, n_interactions=n_interactions,
cutoff=cutoff , cutoff_network=spk.nn.cutoff.CosineCutoff
)

# Define SchNet output model and property to be predicted

output_U0 = spk.atomistic.Atomwise(n_in=n_filters, atomref=atomrefs[QM9.U0])

# Define atomistic model

model = spk.AtomisticModel(representation=schnet,output_modules=output_U0)

# Load saved checkpoint file
load_checkpoint = torch.load(checkpoint_path)


#qm9_i6_30f_20g-1000-500-4_300.pth
# load model's state dictionary from saved checkpoint
model.load_state_dict(load_checkpoint)


#set up device for forward pass
device='cpu'

# load spk calculator
calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.U0)
converter = spk.data.AtomsConverter(device=device)

vec = np.zeros((1,30))


inputs = converter(atoms)

    
layer = None

model.representation.embedding.register_forward_hook(hook)
model(inputs)

emb = layer.clone()
emb = layer.detach().numpy()

layer = None

model.representation.interactions[0].register_forward_hook(hook)

model(inputs)
int0 = layer.clone()
int0 = int0.detach().numpy()    

layer = None

model.representation.interactions[1].register_forward_hook(hook)

model(inputs)
int1 = layer.clone()
int1 = int1.detach().numpy()   

layer = None 

model.representation.interactions[2].register_forward_hook(hook)

model(inputs)   

int2 = layer.clone()
int2 = int2.detach().numpy()   

rep = emb+int0+int1+int2


rows = np.zeros((number_atoms,30))
for i in range(number_atoms):
    for j in range(30):
        rows[i][j] = rep[0][i][j]

from schnetpack.atomistic.output_modules import yi

yi=yi.detach().numpy()

for i in range(number_atoms):
    vec = np.vstack((vec,rows[i]))


save_data_file = '../../data/datasets/extendchain/%s.csv' %(name)
savetxt(save_data_file,vec,delimiter=',') 